In [4]:
# .env
# configured with DEEP_GLACIER as storage class
# but could also work with GlACIER storage class
%env REMOTE_AWS_DEEP=aws_deep
%env ROOT_AWS=test-glacier-restore-htr
%env _rclone=./rclone --dry-run

env: REMOTE_AWS_DEEP=aws_deep
env: ROOT_AWS=test-glacier-restore-htr
env: _rclone=./rclone --dry-run


In [2]:
%%bash
#set up AWS root
rclone purge ${REMOTE_AWS_DEEP}:${ROOT_AWS}
mkdir test && touch $_/{hello-world.txt,abc,c}
mkdir -p test/long/path/empty/dir test/some/long/path && touch $_/{a,b}
rclone ls test
rclone copy test ${REMOTE_AWS_DEEP}:${ROOT_AWS}/test
rm -rf test

2022/06/14 21:23:54 Failed to create file system for ":": config name needs a trailing `:`


        0 abc
        0 c
        0 hello-world.txt
        0 some/long/path/a
        0 some/long/path/b


2022/06/14 21:23:54 Failed to create file system for ":/test": config name contains invalid characters - may only contain `0-9`, `A-Z`, `a-z`, `_`, `-`, `.` and space


# Using `rclone backend restore`

`rclone backend restore <remote:>path/to/restore/dir -o lifetime=<N> -o priority=<Standard|Expedite|Bulk>`
- `N` is the time in days the restored object will be in higher tier for restoration
- `Priority` refers to how urgent the restore request is.

In [20]:
%%bash
rclone backend help ${REMOTE_AWS_DEEP}: | sed '/^$/d' | tail -n +10 | head -n 10

### restore
Restore objects from GLACIER to normal storage
    rclone backend restore remote: [options] [<arguments>+]
This command can be used to restore one or more objects from GLACIER
to normal storage.
Usage Examples:
    rclone backend restore s3:bucket/path/to/object [-o priority=PRIORITY] [-o lifetime=DAYS]
    rclone backend restore s3:bucket/path/to/directory [-o priority=PRIORITY] [-o lifetime=DAYS]
    rclone backend restore s3:bucket [-o priority=PRIORITY] [-o lifetime=DAYS]
This flag also obeys the filters. Test first with -i/--interactive or --dry-run flags


In [19]:
%%bash
$_rclone backend restore ${REMOTE_AWS_DEEP}:${ROOT_AWS}/test/some -o lifetime=3 -o priority=Standard

[
	{
		"Status": "OK",
		"Remote": "long/path/a"
	},
	{
		"Status": "OK",
		"Remote": "long/path/b"
	}
]


## Tips: Use [filters](./standard_ops.ipynb#selections--filtering) to select files to restore

Per the documentation, the restore comand respects filters

In [26]:
!rclone backend help aws: | sed '/^$/d' | tail -n +20 | head -n 3

    rclone -i backend restore --include "*.txt" s3:bucket/path -o priority=Standard
All the objects shown will be marked for restore, then
    rclone backend restore --include "*.txt" s3:bucket/path -o priority=Standard


In this example, we have 2 files in same folder: `bucket/test/long/path/{a,b}`. We will restore only `b`.

In [36]:
%%bash
$_rclone --include "b" backend restore ${REMOTE_AWS_DEEP}:${ROOT_AWS}/test/some/long/path

2022/06/14 21:15:45 NOTICE: b: Skipped restore as --dry-run is set (size 0)


[
	{
		"Status": "OK",
		"Remote": "b"
	}
]


## Nitpick: The arguments are required

Without either of the `-o priority=<N> -o lifetime=<..>`, we will get a hard-to-debug message.

This is due to the S3 Go (and many other languages) SDK doesn't handle this requirement and fail quick.

NOTE: if you do `--dry-run` here, there will be no error message in the status.
This is because `rclone` is blind of these bits.

In [23]:
%%bash
rclone backend restore ${REMOTE_AWS_DEEP}:${ROOT_AWS}/test | head -n 5

[
	{
		"Status": "MalformedXML: The XML you provided was not well-formed or did not validate against our published schema\n\tstatus code: 400, request id: 7J69AF7DVYG39X14, host id: cqQrm03eQzTG/ZRN4XTrqBVZbtU9QwJHQC/sP2qZKCz6KKfXN6YSriPQc9B+shv35qcdMM/Lq34=",
		"Remote": "abc"
	},


## Nitpick: The path must point to a directory (or a bucket)

Because of implementation details, `rclone` couldn't restore by object (file)
despite the documentation includes `path/to/object`.

In [38]:
%%bash
$_rclone backend restore ${REMOTE_AWS_DEEP}:${ROOT_AWS}/test/hello-world.txt -o lifetime=3 -o priority=Standard
echo "Exited: $?"

2022/06/14 21:17:12 Failed to backend: is a file not a directory


Exited: 1


In [5]:
%%bash
# Clear the bucket footprint
rclone purge ${REMOTE_AWS_DEEP}:/${ROOT_AWS}